Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [79]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [80]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [81]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [82]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [83]:
batch_size = 16 #Number of epochs
patch_size = 5 #Kernel size
depth = 16 #Output layer after convolution
num_hidden = 64  #Number of neurons in deep layer

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))    
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    print(shape)
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print(reshape.get_shape().as_list())
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 7, 7, 16]
[16, 784]
[10000, 7, 7, 16]
[10000, 784]
[10000, 7, 7, 16]
[10000, 784]


In [77]:
num_steps = 101

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized


IndexError: too many indices

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [76]:
batch_size = 16
depth = 16
patch = 5
num_output = 64

g = tf.Graph()
with g.as_default():
    def weight_initialize(weight_size):
        weights_ready = tf.random_normal(shape=weight_size, stddev=0.1)
        return tf.Variable(weights_ready)

    def bias_initialize(bias_size):
        bias_ready = tf.constant(0.5,shape=bias_size)
        return tf.Variable(bias_ready)

    with open('notMNIST.pickle','rb') as f:
        data = pickle.load(f)
        train_dataset = data["train_dataset"]
        train_labels = data["train_labels"]
        valid_dataset = data["valid_dataset"]
        valid_labels = data["valid_labels"]
        test_dataset = data["test_dataset"]
        test_labels = data["test_labels"]

    tf_train_data =tf.placeholder(tf.float32,shape = (batch_size, image_size, image_size), name = None)
    tf_labels = tf.placeholder(tf.float32,shape = (batch_size, num_labels), name = None)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    #Conv layer1 - Here the choice of weight dimensions does not depend on input size. 
    #But based on input, kernel, padding and stride - op size is computed
    layer1_w = weight_initialize([patch,patch,num_channels, depth])
    layer1_b = bias_initialize([depth])
    #Conv layer 2- Here the choice of weight dimensions does not depend on input size. 
    #But based on input, kernel, padding and stride - op size is computed
    layer2_w = weight_initialize([patch,patch,depth, depth])
    layer2_b = bias_initialize([depth])
    #Hidden layer - hidden neurons is 64 here
    layer3_w = weight_initialize([image_size // 4 * image_size //  4 *depth , num_output])
    layer3_b = bias_initialize([num_output])
    #Readout layer
    layer4_w = weight_initialize([num_output, num_labels])
    layer4_b = bias_initialize([num_labels])
    
    def train_model(inp_data):
        #Layer1
        op_1 = tf.nn.conv2d(inp_data, layer1_w, 
                            strides = [1,1,1,1], padding = 'SAME') #Convolution
        relu_1 = tf.nn.relu(op_1 + layer1_b)                       #Relu
        conv_1 = tf.nn.max_pool(relu_1,ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')      #Max Pooling
        
        #Layer2
        op_2 = tf.nn.conv2d(conv_1, layer2_w, 
                            strides = [1,1,1,1], padding = 'SAME') #Convolution
        relu_2 = tf.nn.relu(op_2 + layer2_b)                       #Relu
        conv_2 = tf.nn.max_pool(relu_2,ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')      #Max Pooling
        
        #Layer3
        #Reshaping to suit the fully connected layer
        conv2_size = conv_2.get_shape().as_list()
        reshape_2 = tf.reshape(conv2_size,[conv2_size[0],conv2_size[1]*conv2_size[2]*conv2_size[3]])
        
        op_3 = (tf.matmul(reshape_2,layer3_w) + layer3_b)
        relu_3 = tf.nn.relu(op_3)
        
        #Layer4
        op_4 = (tf.matmul(relu_3,layer4_w) + layer4_b)
        
        return op_4
    
    #Run the above function for a batch size and return the outputs
    logits = train_model(tf_train_data)
    
    #Loss computation for that batch size (reduce the mean for the average of those inputs for one step)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits,tf_labels))
         
    #Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    #Evaluate the performance for training data (occassionally), validation and testData towards the end
    tf_train_predict = tf.nn.softmax(logits)
    tf.valid_predict = tf.nn.softmax(model(tf_valid_dataset))
    tf.test_predict = tf.nn.softmax(model(tf_test_dataset))

In [74]:
#Actual graph execution
num_steps = 1001

with tf.Session(graph = g) as sess:
  tf.initialize_all_variables()
  
  for step in range(num_steps)
    offset = (step)*batch_size
    
    tf_train_data = train_dataset([offset:offset+batch_size,:,:])
    tf_train_labels = train_labels([offset:offset+batch_size])
    feed_dict = {tf_train_data : tf_train_data, tf_labels : tf_train_labels}
    
    #Ouput arguments order is same as input 'fetches' arguments
    _,l,prediction = sess.run([optimizer,loss,tf_train_predict],feed_dict)
    if (step%50) == 0
      acc = accuracy (prediction,tf_train_labels)
      print("Minibatch accuracy at step %d:%f", step,acc)
    
  valid_pred = sess.run(tf.valid_predict)
  valid_acc = accuracy(valid_pred,tf_valid_labels)
  test_pred = sess.run(tf.test_predict)
  test_acc = accuracy(test_pred,tf_test_labels)
  printf("Validation accuracy = %f", valid_acc)
  printf("Test accuracy = %f", test_acc)

784


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---